In [45]:
import re

default = "The condor, cougar and serpent represent the three parts of our world. Each level contains a special symbol and specific purpose. It is our duty to understand, learn and protect our three symbols. For man to reach his fullest capacity, he must go through all three levels. Man needs to shed like a snake and become steady as a cougar. Then, man will be able to reach freedom as the condor."
dictionary = {}
r = round
l = len
sim_thresh = 0.5
debug = True

def jaccard(word, other):
    i = len(set(word).intersection(set(other)))
    u = len(set(word).union(set(other)))
    return i/u

def read_dict(filename, dictionary):
    n=len(dictionary)
    with open(filename+".csv") as fp:
        for line in fp:
            line = line.lower().strip().split(',')
            if line[0] not in dictionary:
                dictionary [line[0].strip()] = [line[2],line[1]]
                n+=1
        if len(dictionary) == n: print(f"Message: no duplicate entries in {filename}.")
        else: print(f"Warning: duplicate entries detected in {filename}: {n -len(dictionary)}")
    return dictionary

def is_noun(word):
    try:
        noun = dictionary[word][1] == "noun"
    except:
        pl = False
        if word[-1] == "s":
            word = word[:-1]
            pl = True
            
        try:
            noun = dictionary[word][1] == "noun"
        except:
            if pl:
                word += "s"
            word_approx = re.search("\(\w+,",find_similar(word, dictionary)).group()[1:-1]
            noun = dictionary[word_approx][1] == "noun"
    return noun

def is_verb(word):
    try:
        verb = dictionary[word][1] == "verb"
    except:
        p2 = False
        if word[-1] == "s":
            word = word[:-1]
            p2 = True
        try:
            verb = dictionary[word][1] == "verb"
        except:
            if p2:
                word += "s"
            word_approx = re.search("\(\w+,",find_similar(word, dictionary)).group()[1:-1]
            verb = dictionary[word_approx][1] == "verb"
    return verb

def is_plural(word):
    return is_noun(word) and word[-1] == "s"

def object_casing(word_obj):
    if word_obj["noun"]:
        suffixes = {"with": "wan", #instrumental
                    "without": "naq", #abessive
                    "for": "paq", #dative
                    "to": "paq", #dative
                    "of": "pa", #genitive
                    "than": "hina", #comparative
                    "as": "hina", #comparative
                    "like": "hina" #comparative
        }
        for preposition in word_obj["context"][::-1]:
            if preposition in suffixes:
                word_obj["tail"] = suffixes[preposition] + word_obj["tail"]
                return word_obj
        if word_obj["object"]: word_obj["tail"] = "ta" + word_obj["tail"] #accusative
    return word_obj

def verb_conjugation(word_obj):
    if word_obj["verb"] and "to" not in word_obj["context"]:
        person = ""
        plural = ""
        suffixes = [
            {
                "1":"ni",
                "2":"nqi",
                "3":"n",
            },
            {
                "1":"cqu",
                "2":"nqu",
                "3":"nqu"
            }
        ]
        tenses = {
            "will":"saq",
            "was":"rqa"
        }
        noun_cnt = 0
        for item in word_obj["context"]:
            noun_cnt += is_noun(item)
        if noun_cnt > 1 or "we" in word_obj["context"] or "they" in word_obj["context"]:
            word_obj["plural"] = True
            if "he" in word_obj["context"] or "she" in word_obj["context"]:
                word_obj["plural"] = False
        
        if "i" in word_obj["context"] or "we" in word_obj["context"]: person = "1"
        elif "you" in word_obj["context"]: person = "2"
        else: person = "3"
        word_obj["tail"] = suffixes[word_obj["plural"]][person] + word_obj["tail"]
        for indic in word_obj["context"][::-1]:
            if indic in tenses:
                word_obj["tail"] = tenses[indic] + word_obj["tail"]
    return word_obj

def find_similar(word, dictionary):
    max_sim = 0
    for other in dictionary:
        new_sim = jaccard(word, other)
        if new_sim > max_sim:
            winner, max_sim = other, new_sim
    if max_sim > sim_thresh: qic_word = dictionary[winner][0] + f" ({winner},{r(max_sim,2)})"
    else: qic_word = word + f" ({winner},{r(max_sim,2)})"
    return qic_word

def preprocess(word, word_obj):
    word_obj["orig"] = word
    
    try: word_obj["header"] = re.search("^[^a-z]+\w", word).group()[:-1]
    except: word_obj["header"] = ''
    try: word_obj["tail"] = re.search("\w[^a-z]+", word).group()[1:]
    except: word_obj["tail"] = ''
    try: word_obj["word"] = re.search("\w+", word).group()
    except: word_obj["word"] = word
        
    word_obj["noun"] = is_noun(word_obj["word"])
    word_obj["verb"] = is_verb(word_obj["word"])
    word_obj["plural"] = is_plural(word_obj["word"])
    
    word_obj["tail"] = "quna"*word_obj["plural"] + word_obj["tail"]
    if word_obj["verb"]: word_obj["object"] = 1
    word_obj = object_casing(word_obj)
    word_obj = verb_conjugation(word_obj)
    word_obj["context"].append(word_obj["word"])
    if l(word_obj["context"]) > 4:
        word_obj["context"] = word_obj["context"][1:]
    if "." in word_obj["tail"]: 
        word_obj["object"] = 0
        word_obj["context"] = []
    return word_obj

def postprocess(word_obj):
    return word_obj["header"] + word_obj["qic_word"] + word_obj["tail"]

def title_case(text):
    prev_word = ""
    out = []
    for word in text.split(" "):
        if word == "": continue
        if prev_word == "": word = word.title()
        if "." in prev_word:    
            word = word.title()
        prev_word = word
        out.append(word)
    return " ".join(out)

def en2qic(raw, dictionary):
    missing_words = []
    truly_missing_words = []
    translation = ""
    used_words = []
    word_obj = {"context": [], "object": 0}
    for word in raw.lower().strip().split(' '):
        word = word.strip()
        if word == '': continue
        word_obj = preprocess(word, word_obj)
        word = word_obj["word"]
        try: qic_word = dictionary[word_obj["word"]][0] + debug*f" ({word},1.00)"
        except: 
            missing_words.append(word_obj["word"])
            if word_obj["word"][-1] == 's':
                word_obj["word"] = word_obj["word"][:-1]
            try:
                qic_word = dictionary[word_obj["word"]][0] + debug*f" ({word[:-1]},1.00)"
            except:
                word_obj["word"] += "s"
                qic_word = find_similar(word_obj["word"], dictionary)
        word_obj["qic_word"] = qic_word.split(" ")[0]
        if word_obj["qic_word"] == word_obj["word"]:
            missing_words.append(word_obj["word"])
        if debug:
            word_obj["tail"] += " "+qic_word.split(" ")[1]
        qic_word = postprocess(word_obj)
        translation += qic_word + " "
        used_words.append([word, qic_word, word_obj["word"], word_obj["tail"]])
    if debug:
        for pair in used_words:
            e = pair[0]
            q = pair[1]
            t = pair[3].split(" ")[0]
            print(e,' '*(15-l(e)),"|",q,t)
        print("Words not in dictionary:", missing_words)
        print("Unable to match these words:", truly_missing_words)
    return translation

dictionary = read_dict("dictionary_az", dictionary)

raw = input()
if raw == "":
    raw = default
print("--------------------------------------------")
    
en2qic(raw, dictionary)
print(raw)
debug = False
print(title_case(en2qic(raw, dictionary)))

Message: no duplicate entries in dictionary_az.
Then, man will be able to reach freedom as the condor.
--------------------------------------------
then             | hinaspa, (then,1.00) ,
man              | runa (man,1.00) 
will             | n (will,1.00) n
be               | ñulaysaqn (be,1.00) saqn
able             | qutisaqn (able,1.00) saqn
to               |  (to,1.00) 
reach            | aypay (reach,1.00) 
freedom          | qispiypaq (freedom,1.00) paq
as               | hina (as,1.00) 
the              |  (the,1.00) 
condor           | qunturhina. (condor,1.00) hina.
Words not in dictionary: []
Unable to match these words: []
Then, man will be able to reach freedom as the condor.
Hinaspa, runa n ñulaysaqn qutisaqn aypay qispiypaq hina qunturhina.
